In [ ]:
#!pip3 uninstall pymysql



In [1]:
!pip3 install pymysql==0.9.3

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached PyMySQL-0.9.3-py2.py3-none-any.whl (47 kB)
  Attempting uninstall: pymysql
    Found existing installation: PyMySQL 1.0.2
    Uninstalling PyMySQL-1.0.2:
      Successfully uninstalled PyMySQL-1.0.2
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [28]:
import pymysql

In [40]:
import boto3
import pandas as pd
import numpy as np
import s3fs

In [90]:
path1='s3://productscsv/toys_rating4.csv'
df=pd.read_csv(path1)
df.head()

,Title,Image,adeclarative,alinknormal_URL,asizebase,aoffscreen,Price,Price1,asizebase2,arow,Like,scouponunclipped__spancontains_class_acolorbase,arow3,acolorbase
0,NaN,NaN,Sponsored\n,NaN,NaN,Shop Fun Creative Gifts for Boys of All Ages,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282,$54.99,54\n.,$59.99,NaN,NaN,NaN,NaN,Get it as soon as,"Sat, Apr 30"
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"4,333",$29.99,29\n.,NaN,NaN,NaN,NaN,NaN,Get it as soon as,"Sat, Apr 30"
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1,$7.99,7\n.,NaN,NaN,NaN,NaN,NaN,Get it as soon as,"Tue, May 3"
4,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,"11,005",$19.99,19\n.,$38.99,NaN,10% coupon applied at checkoutSave 10% with c...,Save 10%,with coupon,Get it as soon as,"Sat, Apr 30"


In [91]:
# limiting these columns
new_df=df[['Title','Image','adeclarative','alinknormal_URL','asizebase','aoffscreen','Price1','arow']] 

In [92]:
#Removing sponsored ad wordings from df
df1=new_df[~new_df.adeclarative.str.contains("Sponsored\n",na=False)]
df1.head()

,Title,Image,adeclarative,alinknormal_URL,asizebase,aoffscreen,Price1,arow
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282,$54.99,$59.99,NaN
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"4,333",$29.99,NaN,NaN
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1,$7.99,NaN,NaN
4,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,"11,005",$19.99,$38.99,10% coupon applied at checkoutSave 10% with c...
5,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,"1,991",$25.97,$35.99,20% coupon applied at checkoutSave 20% with c...


In [93]:
#Renaming columns withn readble columns names
df1=df1.rename(
        columns={
            'adeclarative':'Rating',
            'Image':'Image',
            'alinknormal_URL':'ProdLink',
            'asizebase':'TotalReviews',
            'aoffscreen':'Price',
            'Price1':'OrgPrice',
            'arow':'Coupons'
               
        })

In [95]:
df1.head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282,$54.99,$59.99,NaN
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"4,333",$29.99,NaN,NaN
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1,$7.99,NaN,NaN
4,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,"11,005",$19.99,$38.99,10% coupon applied at checkoutSave 10% with c...
5,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,"1,991",$25.97,$35.99,20% coupon applied at checkoutSave 20% with c...


In [96]:
df = df1.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [97]:
df.tail()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
171,LEGO Speed Champions Aston Martin Valkyrie AMR...,https://m.media-amazon.com/images/I/818L58F+ei...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,65,$39.97,NaN,NaN
172,Dinosaur Toy Truck for Kids 3-7 with Flash Lig...,https://m.media-amazon.com/images/I/81aDx6A2XT...,4.5 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,78,$24.99,NaN,NaN
173,Paw Patrol Marshall - Robotic Build-A-Bot Paw ...,https://m.media-amazon.com/images/I/718M5NdTF7...,4.1 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"2,163",$34.99,NaN,NaN
174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
cols = df1.select_dtypes(['object']).columns
df1[cols] = df1[cols].apply(lambda x: x.str.strip())
#print(df1.head())
df1.tail()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
171,LEGO Speed Champions Aston Martin Valkyrie AMR...,https://m.media-amazon.com/images/I/818L58F+ei...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,65,$39.97,NaN,NaN
172,Dinosaur Toy Truck for Kids 3-7 with Flash Lig...,https://m.media-amazon.com/images/I/81aDx6A2XT...,4.5 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,78,$24.99,NaN,NaN
173,Paw Patrol Marshall - Robotic Build-A-Bot Paw ...,https://m.media-amazon.com/images/I/718M5NdTF7...,4.1 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"2,163",$34.99,NaN,NaN
174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
#taking only not null from image, prodlink,title..basically removing null values from title 
df = df1[df1['Image'].notna()]
df = df1[df1['ProdLink'].notna()]
df = df1[df1['Title'].notna()]
df.head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282,$54.99,$59.99,NaN
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"4,333",$29.99,NaN,NaN
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1,$7.99,NaN,NaN
4,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,"11,005",$19.99,$38.99,10% coupon applied at checkoutSave 10% with c...
5,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,"1,991",$25.97,$35.99,20% coupon applied at checkoutSave 20% with c...


In [100]:
#Handling Nan in Total Reviews, replacing with 0
df['TotalReviews'] = df['TotalReviews'].replace(np.nan, 0)

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/2447984689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalReviews'] = df['TotalReviews'].replace(np.nan, 0)


In [101]:
#Fill Missing Values of price and original price vice versa
df.Price.fillna(df.OrgPrice, inplace=True)
df.OrgPrice.fillna(df.Price, inplace=True)
df.head()

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/1399759672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Price.fillna(df.OrgPrice, inplace=True)
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/1399759672.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.OrgPrice.fillna(df.Price, inplace=True)


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282,$54.99,$59.99,NaN
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"4,333",$29.99,$29.99,NaN
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1,$7.99,$7.99,NaN
4,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,"11,005",$19.99,$38.99,10% coupon applied at checkoutSave 10% with c...
5,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,"1,991",$25.97,$35.99,20% coupon applied at checkoutSave 20% with c...


In [102]:
#Print Missing Values for reference
#coupon column, NAN
df[df.isnull().any(axis=1)].head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282,$54.99,$59.99,NaN
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,"4,333",$29.99,$29.99,NaN
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1,$7.99,$7.99,NaN
10,CENOVE Girls Toys Age 3-6 Year Old Toddler Toy...,https://m.media-amazon.com/images/I/71SL3Pv2u4...,4.8 out of 5 stars,https://www.amazon.com/CENOVE-Toddler-Flower-B...,"5,096",$35.99,$49.99,NaN
13,"Kids Drawing Writing Board Doodle Pad, DELLAN ...",https://m.media-amazon.com/images/I/61lvc0pW+T...,4.5 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,29,$11.99,$11.99,NaN


In [103]:
#Removing Commas in the  Total Reviews and converting them to float
df['TotalReviews'] = df['TotalReviews'].str.replace(',', '').astype(float)
df.tail()

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/3601446635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalReviews'] = df['TotalReviews'].str.replace(',', '').astype(float)


,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
169,ECTY Unicorn Gifts Lamps Night Lights for Kids...,https://m.media-amazon.com/images/I/81gn4sWprG...,4.4 out of 5 stars,https://www.amazon.com/LONGXU-Unicorn-Changing...,2089.0,$15.99,$26.99,Lowest price in 30 days
170,"The Original Slinky Walking Spring Toy, Metal ...",https://m.media-amazon.com/images/I/91jLIfnfDn...,4.5 out of 5 stars,https://www.amazon.com/Slinky-60100-Original-B...,23933.0,$3.50,$3.50,50% off gift wrap service: code GIFTWRAP50
171,LEGO Speed Champions Aston Martin Valkyrie AMR...,https://m.media-amazon.com/images/I/818L58F+ei...,4.8 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,65.0,$39.97,$39.97,NaN
172,Dinosaur Toy Truck for Kids 3-7 with Flash Lig...,https://m.media-amazon.com/images/I/81aDx6A2XT...,4.5 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,78.0,$24.99,$24.99,NaN
173,Paw Patrol Marshall - Robotic Build-A-Bot Paw ...,https://m.media-amazon.com/images/I/718M5NdTF7...,4.1 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,2163.0,$34.99,$34.99,NaN


In [104]:
df.dtypes

Title            object
Image            object
Rating           object
ProdLink         object
TotalReviews    float64
Price            object
OrgPrice         object
Coupons          object
dtype: object

In [105]:
import locale
#removing $ from price
locale.setlocale(locale.LC_ALL,'')
df['Price']=df.Price.map(lambda x: locale.atof(x.strip('$')))
df['OrgPrice']=df.OrgPrice.map(lambda x: locale.atof(x.strip('$')))

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/1767316431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price']=df.Price.map(lambda x: locale.atof(x.strip('$')))
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/1767316431.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OrgPrice']=df.OrgPrice.map(lambda x: locale.atof(x.strip('$')))


In [106]:
df.head()

,Title,Image,Rating,ProdLink,TotalReviews,Price,OrgPrice,Coupons
1,Cats vs Pickles - Limited Edition Holiday 12pk...,https://m.media-amazon.com/images/I/51n8ENJFUW...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,282.0,54.99,59.99,NaN
2,Osmo - Super Studio Disney Frozen 2 - Ages 5-1...,https://m.media-amazon.com/images/I/91SJQup1fq...,4.7 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,4333.0,29.99,29.99,NaN
3,Pop it Pink Ombre Cupcake Fidget Toy Girls and...,https://m.media-amazon.com/images/I/81G2b5S-wQ...,5.0 out of 5 stars,https://www.amazon.com/gp/slredirect/picassoRe...,1.0,7.99,7.99,NaN
4,"Jasonwell Splash Pad Sprinkler for Kids 68"" Sp...",https://m.media-amazon.com/images/I/81s0fdVXUu...,4.4 out of 5 stars,https://www.amazon.com/Jasonwell-Sprinkle-Spri...,11005.0,19.99,38.99,10% coupon applied at checkoutSave 10% with c...
5,Magnetic Tiles Toys for 3 4 5 6 7 8+ Year Old ...,https://m.media-amazon.com/images/I/71DQtErS2T...,4.6 out of 5 stars,https://www.amazon.com/Magnetic-Building-Creat...,1991.0,25.97,35.99,20% coupon applied at checkoutSave 20% with c...


In [107]:
df['Price'] = pd.to_numeric(df['Price'])
df['OrgPrice'] = pd.to_numeric(df['OrgPrice'])

/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/2507014352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = pd.to_numeric(df['Price'])
/var/folders/3c/hxpmknwd5cg9lwydrkf402qw0000gp/T/ipykernel_10261/2507014352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OrgPrice'] = pd.to_numeric(df['OrgPrice'])


In [108]:
df.dtypes

Title            object
Image            object
Rating           object
ProdLink         object
TotalReviews    float64
Price           float64
OrgPrice        float64
Coupons          object
dtype: object

# Connect to AWS RDS 

In [109]:
connection = pymysql.connect( 'database-1.cbrnhoat32p7.us-east-1.rds.amazonaws.com', 'myuser', 'myPass05')
cursor = connection.cursor()
print(cursor)

In [4]:
cursor.execute("select version()")


1

In [5]:
data = cursor.fetchone()

In [6]:
data

('8.0.28',)

In [7]:
sql = '''select database collections'''

In [8]:
cursor.connection.commit()

In [9]:
sql = '''use collections'''  # use the db
cursor.execute(sql)

0

In [10]:
#show the tables in RDs
sql = '''
show tables
'''
cursor.execute(sql)


1

In [11]:
cursor.fetchall()

(('items',),)

In [12]:
sql = '''select * from items'''
cursor.execute(sql)
cursor.connection.commit()